In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import ICA
import pywt
from scipy import signal
from scipy.interpolate import Rbf
from PIL import Image
from tqdm import tqdm
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")
mne.set_log_level("WARNING") 

In [ ]:
import os
import shutil

def clear_directory(directory_path):
    if not os.path.exists(directory_path):
        print(f"Directory does not exist: {directory_path}")
        return
    
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete file or symbolic link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete directory and its contents
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

directory_to_clear = "/kaggle/working/"
clear_directory(directory_to_clear)
print(f"Cleared all contents in: {directory_to_clear}")

In [3]:
label_num = {'Seizure': 0, 'GPD': 1, 'LPD': 2, 'GRDA': 3, 'LRDA': 4, 'Other': 5}
channel_names = ['Fp1','F3','C3','P3','F7','T3','T5','O1','Fz','Cz','Pz','Fp2','F4','C4','P4','F8','T4','T6','O2']
channel_types = ['eeg'] * len(channel_names)
sfreq = 200

channel_indices = {name: idx for idx, name in enumerate(channel_names)}

grid_structure = [
    [0,    'Fp1', 0,    'Fp2', 0],
    ['F7', 'F3',  'Fz', 'F4',  'F8'],
    ['T3', 'C3',  'Cz', 'C4',  'T4'],
    ['T5', 'P3',  'Pz', 'P4',  'T6'],
    [0,    'O1',  0,    'O2',  0]
]

In [4]:
def signal_processing(a,b):
    eeg = pd.read_parquet("/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"+str(a)+".parquet")
    eeg = eeg[int(b)*200:int(50+b)*200] 
    eeg.fillna(eeg.mean(), inplace = True) 
    eeg_data = eeg.iloc[:,:-1].to_numpy().T  
    
    info = mne.create_info(ch_names = channel_names, sfreq = sfreq, ch_types = channel_types)
    raw = mne.io.RawArray(eeg_data, info)
    std_montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(std_montage)

    raw.notch_filter(60)

    raw.filter(l_freq = 1, h_freq = 50, method = 'iir', iir_params = None)

    raw_data = raw.get_data() 
    raw_times = raw.times
    info = raw.info 
    normalized_eeg = mne.baseline.rescale(raw_data, raw_times, baseline = (None, None), mode = 'mean')
    normalized_eeg = mne.io.RawArray(normalized_eeg, info)

    return normalized_eeg

In [5]:
def wavelet_enhanced_ica(normalized_eeg):
    ica = mne.preprocessing.ICA(n_components = 19, method = 'infomax', fit_params = dict(extended=True), max_iter = 'auto')
    ica.fit(normalized_eeg.copy())
    
    data = normalized_eeg.get_data()
    mixing_matrix = ica.mixing_matrix_
    unmixing_matrix = ica.unmixing_matrix_
    pca_components = ica.pca_components_
    pca_mean = ica.pca_mean_
    n_pca_components = ica.n_components_

    data_centered = data - pca_mean[:, np.newaxis]
    pca_data = np.dot(pca_components[:n_pca_components], data_centered)
    ica_sources = np.dot(unmixing_matrix, pca_data)

    denoised_components = np.zeros(data.shape)
    wavelet_family = 'db4'
    resolution = 5
    
    for i in range(19):
        coefficients = pywt.wavedec(ica_sources[i], wavelet = wavelet_family, level = resolution)
        sigma = (1/0.6745) * np.median(np.abs(coefficients[5] - np.median(coefficients[5]))) 
        threshold = sigma * np.sqrt(2 * np.log(len(coefficients[5])/sfreq)) 
        coefficients[1:] = [np.where(np.abs(coeff) > threshold, 0, coeff) for coeff in coefficients[1:]]
        denoised_components[i] = pywt.waverec(coefficients, wavelet = wavelet_family)
        
    ica_reconstructed = np.dot(mixing_matrix, denoised_components)
    reconstructed_data = np.dot(pca_components[:n_pca_components].T, ica_reconstructed)
    reconstructed_data += pca_mean[:, np.newaxis] # numpy.ndarray:shape(19, 10000):data.shape

    return reconstructed_data # (n_channels, n_samples)

In [6]:
def feature_maps(eeg_data):
    representation_matrix = np.zeros((10000, 5, 5))
    for t in range(10000):
        for i in range(5):
            for j in range(5):
                channel = grid_structure[i][j]
                if channel != 0: 
                    representation_matrix[t, i, j] = eeg_data[channel_indices[channel], t]
    
    rbf_interpolated_data = np.zeros_like(representation_matrix)
    x_grid, y_grid = np.meshgrid(np.arange(5), np.arange(5))
    known_positions = []
    known_values = []

    for t in range(10000):
        known_positions = []
        known_values = []
        for i in range(5):
            for j in range(5):
                if grid_structure[i][j] != 0:  
                    known_positions.append([i, j])
                    known_values.append(representation_matrix[t, i, j])
        known_positions = np.array(known_positions)
        known_values = np.array(known_values)
        if len(known_values) == 0:
            continue
        rbf = Rbf(known_positions[:, 0], known_positions[:, 1], known_values, function='gaussian')
        for i in range(5):
            for j in range(5):
                if grid_structure[i][j] == 0: 
                    rbf_interpolated_data[t, i, j] = rbf(i, j)
                else: 
                    rbf_interpolated_data[t, i, j] = representation_matrix[t, i, j]
                    
    # feature_maps = np.zeros((10000, 64, 64))
    # dim = (64, 64)
    # for i in range(10000):
    #     image = Image.fromarray(rbf_interpolated_data[i])
    #     resized_map = image.resize(dim, Image.Resampling.BICUBIC)
    #     resized_map_array = np.array(resized_map)
    #     feature_maps[i] = resized_map_array

    # extended_feature_maps = np.expand_dims(feature_maps, axis=1)

    extended_feature_maps = np.expand_dims(rbf_interpolated_data, axis=1)
    
    window_size = 5
    T, C, H, W = extended_feature_maps.shape
    assert T % window_size == 0, "Time dimension must be divisible by window_size"
    T_new = T // window_size
    reduced_feature_map = np.median(extended_feature_maps.reshape(T_new, window_size, C, H, W), axis=1)
    
    normalized_feature_map = np.empty_like(reduced_feature_map, dtype=np.float32)
    
    for i in range(reduced_feature_map.shape[0]):
        img = reduced_feature_map[i, 0]
        min_val = np.min(img)
        max_val = np.max(img)
        if max_val > min_val:
            norm_img = (img - min_val) / (max_val - min_val)
        else:
            norm_img = img  
        normalized_feature_map[i, 0] = norm_img
        
    return normalized_feature_map

In [7]:
# train_df = pd.read_csv('/kaggle/input/csv-files/train_sampled.csv', usecols=['eeg_id', 'eeg_label_offset_seconds', 'eeg_sub_id', 'expert_consensus'])

# total_rows = len(train_df)

# split1 = total_rows // 6
# split2 = 2 * split1
# split3 = 3 * split1
# split4 = 4 * split1
# split5 = 5 * split1

# df_part1 = train_df.iloc[:split1]
# df_part2 = train_df.iloc[split1:split2]
# df_part3 = train_df.iloc[split2:split3]
# df_part4 = train_df.iloc[split3:split4]
# df_part5 = train_df.iloc[split4:split5]
# df_part6 = train_df.iloc[split5:]


# output_dir = '/kaggle/working/Train_5' 
# os.makedirs(output_dir, exist_ok = True)

In [7]:
test_df = pd.read_csv('/kaggle/input/csv-files/test_sampled.csv', usecols=['eeg_id', 'eeg_label_offset_seconds', 'eeg_sub_id', 'expert_consensus'])

total_rows_test = len(test_df)
split_test = total_rows_test // 2

df_test_part1 = test_df.iloc[:split_test]
df_test_part2 = test_df.iloc[split_test:]

output_dir_test = '/kaggle/working/Test_1' 
os.makedirs(output_dir_test, exist_ok=True)

In [10]:
# import logging

# mne_logger = logging.getLogger('mne')
# mne_logger.setLevel(logging.WARNING)

# tqdm_kwargs = {
#     'desc': 'Processing EEGs',
#     'leave': False,  
#     'dynamic_ncols': True, 
#     'bar_format': '{l_bar}{bar}| {n_fmt}/{total_fmt}'  
# }

In [9]:
# # test_df = pd.read_csv('/kaggle/input/csv-files/test_sampled.csv', usecols=['eeg_id', 'eeg_label_offset_seconds', 'eeg_sub_id', 'expert_consensus'])

# def process_eeg(eeg_id, df, output_dir, label_num):
#     eeg_data = df[df["eeg_id"] == eeg_id]
#     for _, row in eeg_data.iterrows():
#         a = row["eeg_id"]
#         b = int(row["eeg_label_offset_seconds"])
#         c = row["eeg_sub_id"]
#         label = label_num[row['expert_consensus']]
        
#         normalized_eeg = signal_processing(a, b)
#         artifact_corrected_data = wavelet_enhanced_ica(normalized_eeg)  # (19, 10000)
#         reduced_feature_maps = feature_maps(artifact_corrected_data)    # (2000, 1, 5, 5)

#         np.savez(f"{output_dir}/{a}_{c}_feature_maps.npz", features = reduced_feature_maps, label = np.array([label]))

# Parallel(n_jobs=4)(delayed(process_eeg)(eeg_id, df_part1, output_dir, label_num) for eeg_id in tqdm(df_part1["eeg_id"].unique(), **tqdm_kwargs))

In [ ]:
import os
import sys
from tqdm import tqdm
from joblib import Parallel, delayed
import logging
import mne

mne.set_log_level(verbose="ERROR")  
logging.getLogger('mne').setLevel(logging.ERROR)
logging.getLogger('joblib').setLevel(logging.ERROR)

def suppress_output(func):
    def wrapper(*args, **kwargs):
        # Redirect stdout/stderr to devnull
        original_stdout = sys.stdout
        original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
        
        try:
            result = func(*args, **kwargs)
        finally:
            # Restore stdout/stderr
            sys.stdout = original_stdout
            sys.stderr = original_stderr
        return result
    return wrapper

@suppress_output
def process_eeg(eeg_id, df, output_dir, label_num):
    eeg_data = df[df["eeg_id"] == eeg_id]
    for _, row in eeg_data.iterrows():
        a = row["eeg_id"]
        b = int(row["eeg_label_offset_seconds"])
        c = row["eeg_sub_id"]
        label = label_num[row['expert_consensus']]
        
        normalized_eeg = signal_processing(a, b)
        artifact_corrected_data = wavelet_enhanced_ica(normalized_eeg)  # (19, 10000)
        reduced_feature_maps = feature_maps(artifact_corrected_data)    # (2000, 1, 5, 5)

        np.savez(f"{output_dir}/{a}_{c}_feature_maps.npz", features=reduced_feature_maps, label=np.array([label]))


tqdm_params = {
    'desc': 'Processing EEGs',
    'bar_format': '{l_bar}{bar}| {n_fmt}/{total_fmt} [Elapsed: {elapsed}, Remaining: {remaining}]',
    'leave': False
}

Parallel(n_jobs=4)(
    delayed(process_eeg)(eeg_id, df_test_part1, output_dir_test, label_num) 
    for eeg_id in tqdm(df_test_part1["eeg_id"].unique(), **tqdm_params)
)



Processing EEGs:   0%|          | 0/1914 [Elapsed: 00:00, Remaining: ?]

Processing EEGs:   0%|          | 8/1914 [Elapsed: 00:19, Remaining: 1:17:09]

In [9]:
!zip -r Train_4.zip Train_4

  adding: Train_4/ (stored 0%)
  adding: Train_4/3562276716_0_feature_maps.npz (deflated 15%)
  adding: Train_4/3258218061_5_feature_maps.npz (deflated 14%)
  adding: Train_4/3123865097_17_feature_maps.npz (deflated 15%)
  adding: Train_4/2830329992_0_feature_maps.npz (deflated 15%)
  adding: Train_4/1136493055_3_feature_maps.npz (deflated 14%)
  adding: Train_4/1562662023_0_feature_maps.npz (deflated 14%)
  adding: Train_4/1454127370_0_feature_maps.npz (deflated 14%)
  adding: Train_4/518192282_0_feature_maps.npz (deflated 15%)
  adding: Train_4/2469408774_3_feature_maps.npz (deflated 15%)
  adding: Train_4/4226704122_12_feature_maps.npz (deflated 15%)
  adding: Train_4/2446242925_1_feature_maps.npz (deflated 14%)
  adding: Train_4/1947784260_0_feature_maps.npz (deflated 15%)
  adding: Train_4/2853295629_5_feature_maps.npz (deflated 15%)
  adding: Train_4/3762078156_1_feature_maps.npz (deflated 15%)
  adding: Train_4/2040599209_0_feature_maps.npz (deflated 15%)
  adding: Train_4/73766

In [10]:
from IPython.display import FileLink 
FileLink(r'Train_4.zip')

/kaggle/working/Train_4.zip